# Bike Share Toronto Data Processing

## Chris Kimber 

The data used comes from the City of Toronto's Open Data Portal (https://open.toronto.ca/). For this series of reports, we are using the data from 2017-2022. I have not provided the dataset on github to avoid any issues involving re-publishing the data, even if it is freely available on the portal. The pipeline below uses the data in the format available for download on the portal in April 2023; the archives were simply uncompressed before use.

Load necessary packages.

In [40]:
library(tidyverse)
library(lubridate)

In [41]:
setwd("../Data/ridership/")

ERROR: Error in setwd("../Data/ridership/"): cannot change working directory


## Import and Cleaning 1: 2019-2022 Data

From an initial inspection of the data, I determined that the format of the data provided by the City of Toronto changes in several aspects over the period from 2017-2022 despite coming from the same data provider according to the city. One major change is in the columns provided in each CSV file. Not all columns are provided each year, column encoding changes, etc. The columns currently provided (as of 2022) have been used for the last 4 years, something I determined from inspecting the data manually. The folders containing CSVs that are produced by unpacking the ZIP archives provided on the Open Data Portal also have a consistent nomenclature over these 4 years (though the CSVs within do not). I will begin by importing these 4 years' worth of data together, and then import the preceding years independently to address their differences in format and make a unified dataset. To import the CSVs for these 4 years, I first find all folders using the relevant naming structure: 

In [42]:
current_format_years <- list.files(path = ".", pattern = "^bikeshare-ridership-[2][0][0-9][0-9]")

Next I find all the csvs of data within these folders, and then read them all in and concatenate them in a single step. From previous inspection of the data I know there are 3 different ways in which missing data is handled in the feature encoding, so I use a custom vector to parse these as NA in readr.

NOTE there is 1 CSV file (Nov 2022) that is within an enclosing ZIP file, unlike all the other CSV in these years. As the format of the CSV naming structure changes year-on-year, it is very difficult to find this automatically and it was uncovered due to manual inspection and unpacked.

In [43]:
current_format_csvs <- list.files(current_format_years, pattern = "*.csv", full.names = TRUE)

In [44]:
current_format_data <- current_format_csvs %>% map(~ read_csv(.x, name_repair = 'universal', na = c("NA", "NULL", ""))) %>% reduce(rbind)

New names:
• `Trip Id` -> `Trip.Id`
• `Trip Duration` -> `Trip..Duration`
• `Start Station Id` -> `Start.Station.Id`
• `Start Time` -> `Start.Time`
• `Start Station Name` -> `Start.Station.Name`
• `End Station Id` -> `End.Station.Id`
• `End Time` -> `End.Time`
• `End Station Name` -> `End.Station.Name`
• `Bike Id` -> `Bike.Id`
• `User Type` -> `User.Type`
Rows: 189063 Columns: 10
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Start.Time, Start.Station.Name, End.Time, End.Station.Name, User.Type
dbl (5): Trip.Id, Trip..Duration, Start.Station.Id, End.Station.Id, Bike.Id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `Trip Id` -> `Trip.Id`
• `Trip Duration` -> `Trip..Duration`
• `Start Station Id` -> `Start.Station.Id`
• `Start Time` -> `Start.Time`
• `Start Station Name` -> `Start.Station.Name`
• `End

In [6]:
head(current_format_data)

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
4581278,1547,7021,01/01/2019 00:08,Bay St / Albert St,7233,01/01/2019 00:33,King / Cowan Ave - SMART,1296,Annual Member
4581279,1112,7160,01/01/2019 00:10,King St W / Tecumseth St,7051,01/01/2019 00:29,Wellesley St E / Yonge St (Green P),2947,Annual Member
4581280,589,7055,01/01/2019 00:15,Jarvis St / Carlton St,7013,01/01/2019 00:25,Scott St / The Esplanade,2293,Annual Member
4581281,259,7012,01/01/2019 00:16,Elizabeth St / Edward St (Bus Terminal),7235,01/01/2019 00:20,Bay St / College St (West Side) - SMART,283,Annual Member
4581282,281,7041,01/01/2019 00:19,Edward St / Yonge St,7257,01/01/2019 00:24,Dundas St W / St. Patrick St,1799,Annual Member
4581283,624,7041,01/01/2019 00:26,Edward St / Yonge St,7031,01/01/2019 00:36,Jarvis St / Isabella St,661,Annual Member


In [7]:
tail(current_format_data)

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
20148774,1961,7706,12/31/2022 23:56,NA,7677,01/01/2023 00:29,1303 Yonge St - SMART,3076,Annual Member
20148775,2825,7374,12/31/2022 23:58,Frederick St / The Esplanade,7048,01/01/2023 00:45,Front St W / Yonge St (Hockey Hall of Fame),788,Casual Member
20148776,1764,7029,12/31/2022 23:58,Bay St / Bloor St W (East Side),7164,01/01/2023 00:27,Gould St / Yonge St (Ryerson University),536,Casual Member
20148777,469,7713,12/31/2022 23:58,NA,7028,01/01/2023 00:06,Gould St / Mutual St,6306,Casual Member
20148782,588,7130,12/31/2022 23:59,Davenport Rd / Bedford Rd,7012,01/01/2023 00:09,Elizabeth St / Edward St (Bus Terminal),3182,Casual Member
20148783,795,7027,12/31/2022 23:59,Beverley St / Dundas St W,7269,01/01/2023 00:13,Toronto Eaton Centre (Yonge St),113,Casual Member


In [8]:
colSums(is.na(current_format_data))

Trip.Id     Trip..Duration   Start.Station.Id         Start.Time 
                 0                 16                  0                  0 
Start.Station.Name     End.Station.Id           End.Time   End.Station.Name 
            187939               5003                  0             193363 
           Bike.Id          User.Type 
               276                249

Inspecting the distribution of NA values, I find some concerning patterns. The high levels of missing data for station names are not themselves concerning as long as station ID is present because the stations remain identifiable. The missing duration and end station data may represent some type of failed trip or trip loggging and will be investigated further, as will the missing bike ID and user type features which have a less obvious a priori explanation.

The missing duration rides are all 0-1 minute rides in 2019. While these rides could be encoded as duration 0 or 1, they also are not informative. Most involve bikes returned immediately at the same station. Several others are returned to stations in close proximity and may even represent true very short rides, while at least one is a clear glitch as the stations are infeasibly far apart for the time stamps given.

In [9]:
current_format_data %>% filter(is.na(current_format_data$Trip..Duration))

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
4651028,NA,7380,01/28/2019 12:45,Erskine Ave / Yonge St SMART,7380,01/28/2019 12:45,Erskine Ave / Yonge St SMART,329,Annual Member
4669491,NA,7309,02/09/2019 21:07,Queen St. E / Rhodes Ave.,7309,02/09/2019 21:07,Queen St. E / Rhodes Ave.,2173,Annual Member
4673600,NA,7100,02/12/2019 13:58,Dundas St E / Regent Park Blvd,7100,02/12/2019 13:59,Dundas St E / Regent Park Blvd,3103,Annual Member
4684203,NA,7324,02/20/2019 09:35,King St W / Charlotte St (West),7324,02/20/2019 09:35,King St W / Charlotte St (West),1834,Annual Member
4724000,NA,7387,03/11/2019 18:47,Mortimer Ave / Carlaw Ave SMART,7387,03/11/2019 18:47,Mortimer Ave / Carlaw Ave SMART,557,Annual Member
4768737,NA,7341,03/23/2019 14:20,Eastern Ave / Winnifred Ave,7341,03/23/2019 14:20,Eastern Ave / Winnifred Ave,3272,Annual Member
4802014,NA,7385,03/31/2019 15:48,20 Charles St E,7385,03/31/2019 15:49,20 Charles St E,3733,Annual Member
4942306,NA,7203,04/26/2019 11:11,Bathurst St/Queens Quay(Billy Bishop Airport),7203,04/26/2019 11:11,Bathurst St/Queens Quay(Billy Bishop Airport),1186,Annual Member
5037597,NA,7354,05/11/2019 13:22,Tommy Thompson Park (Leslie Street Spit),7354,05/11/2019 13:22,Tommy Thompson Park (Leslie Street Spit),714,Annual Member


The missing bike ID rides appear to contain useable data in other features. While they should be excluded when bike ID is a focal feature, they otherwise appear suitable for use.

In [10]:
current_format_data %>% filter(is.na(current_format_data$Bike.Id))

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
10575709,861,7089,12/11/2020 15:43,Church St / Wood St,7163,12/11/2020 15:57,Yonge St / Wood St,NA,Casual Member
10575721,783,7089,12/11/2020 15:44,Church St / Wood St,7163,12/11/2020 15:57,Yonge St / Wood St,NA,Casual Member
10575854,1000,7163,12/11/2020 15:58,Yonge St / Wood St,7542,12/11/2020 16:15,Queen St W / John St,NA,Casual Member
10575855,1002,7163,12/11/2020 15:58,Yonge St / Wood St,7542,12/11/2020 16:15,Queen St W / John St,NA,Casual Member
10575991,1400,7542,12/11/2020 16:15,Queen St W / John St,7259,12/11/2020 16:38,Lower Spadina Ave / Lake Shore Blvd,NA,Casual Member
10575997,1370,7542,12/11/2020 16:15,Queen St W / John St,7259,12/11/2020 16:38,Lower Spadina Ave / Lake Shore Blvd,NA,Casual Member
10576224,1437,7259,12/11/2020 16:38,Lower Spadina Ave / Lake Shore Blvd,7099,12/11/2020 17:02,Cherry St / Mill St,NA,Casual Member
10576226,1395,7259,12/11/2020 16:39,Lower Spadina Ave / Lake Shore Blvd,7099,12/11/2020 17:02,Cherry St / Mill St,NA,Casual Member
10576479,1297,7099,12/11/2020 17:02,Cherry St / Mill St,7089,12/11/2020 17:24,Church St / Wood St,NA,Casual Member


The rides missing end station IDs are apparently a mixed bag with almost 80% of rides being 0 duration and likely reprsenting some sort of failure.

In [11]:
current_format_data %>% filter(is.na(End.Station.Id))

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
5370500,696,7228,06/17/2019 13:21,Queen St W / Roncesvalles Ave,NA,06/17/2019 13:32,NA,2345,Annual Member
5679465,0,7077,07/11/2019 16:45,College Park South,NA,07/11/2019 16:45,NA,1232,Casual Member
6033723,327,7444,08/06/2019 01:41,Clendenan Ave / Rowland St - SMART,NA,08/06/2019 01:46,NA,1890,Casual Member
6735107,979,7432,09/25/2019 14:48,Frederick St / King St E,NA,09/25/2019 15:04,NA,2113,Annual Member
6746731,60,7077,09/26/2019 13:31,College Park South,NA,09/26/2019 13:32,NA,1430,Annual Member
6750357,300,7104,09/26/2019 17:54,King St E / River St,NA,09/26/2019 17:59,NA,3478,Annual Member
6752509,1112,7382,09/26/2019 20:13,Simcoe St / Adelaide St W,NA,09/26/2019 20:32,NA,530,Annual Member
6752610,1620,7076,09/26/2019 20:24,York St / Queens Quay W,NA,09/26/2019 20:51,NA,1557,Annual Member
6754350,1822,7471,09/27/2019 06:32,Lake Shore Blvd W / Louisa St,NA,09/27/2019 07:02,NA,3176,Annual Member


In [35]:
current_format_data %>% filter(is.na(End.Station.Id) & Trip..Duration == "0")

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
5679465,0,7077,07/11/2019 16:45,College Park South,NA,07/11/2019 16:45,NA,1232,Casual Member
6825146,0,7204,10/04/2019 10:46,College St / Crawford St,NA,10/04/2019 10:46,NA,4268,Annual Member
6828893,0,7269,10/04/2019 17:02,Toronto Eaton Centre (Yonge St),NA,10/04/2019 17:02,NA,3210,Annual Member
6828929,0,7118,10/04/2019 17:04,King St W / Bay St (East Side),NA,10/04/2019 17:04,NA,4139,Annual Member
6908875,0,7357,10/11/2019 22:23,Lake Shore Blvd E / Leslie St,NA,10/11/2019 22:23,NA,4910,Annual Member
6914330,0,7458,10/12/2019 17:49,Church St / Lombard St,NA,10/12/2019 17:49,NA,1551,Casual Member
6915256,0,7509,10/12/2019 19:40,Ontario St / King St E,NA,10/12/2019 19:40,NA,2874,Annual Member
6917039,0,7076,10/13/2019 10:31,York St / Queens Quay W,NA,10/13/2019 10:31,NA,1803,Casual Member
7073223,0,7265,10/31/2019 09:04,Wallace Ave / Symington Ave - SMART,NA,10/31/2019 09:04,NA,4880,Annual Member


The missing data for User Type is actually the most interesting, because it also illuminates errors in other factors that cause downstream problems if not detected. Inspection of the rides which are missing the user type information reveals that they all occur in October 2020 but still are just a subset of rides in that period. It appears that in all cases the data on user type appears as the Bike ID instead. Further inspection suggests that all the data has been frameshifted left by one column in these entries, and the root cause is a failure to parse between the Trip ID and Trip Duration features when the City of Toronto generated the CSV. This frameshift obviously generates nonsense data and causes immediate problems in the next step of data ingestion (correcting datatypes).

In [36]:
current_format_data %>% filter(is.na(current_format_data$User.Type))

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
10000084625,7120,10/03/2020 13:28,Gerrard St E / River St,7120,10/03/2020 13:38,Gerrard St E / River St,5250,Annual Member,NA
10000306555,7120,10/03/2020 13:38,Gerrard St E / River St,7576,10/03/2020 13:48,Front St E / Bayview Avenue,5250,Annual Member,NA
10000519608,7576,10/03/2020 13:48,Front St E / Bayview Avenue,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,5250,Annual Member,NA
10000755851,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,5250,Annual Member,NA
10001076784,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,7317,10/03/2020 14:26,Hubbard Blvd / Balsam Av,5250,Annual Member,NA
100013731087,7317,10/03/2020 14:26,Hubbard Blvd / Balsam Av,7318,10/03/2020 14:44,Hubbard Blvd. / Glen Manor Dr.,5250,Annual Member,NA
10001777553,7318,10/03/2020 14:44,Hubbard Blvd. / Glen Manor Dr.,7427,10/03/2020 14:53,Northern Dancer Blvd / Lake Shore Blvd E,5250,Annual Member,NA
100020441499,7427,10/03/2020 14:55,Northern Dancer Blvd / Lake Shore Blvd E,7344,10/03/2020 15:20,Cherry Beach,5250,Annual Member,NA
100026291085,7344,10/03/2020 15:20,Cherry Beach,7261,10/03/2020 15:38,Queens Quay E / Lower Sherbourne St,5250,Annual Member,NA


As the Trip ID feature is sequentially numbered, it should be possible to re-parse these rows and restore their useablilty. For now, given the relatively small amount of data, I exclude it for simplicity's sake.

In [45]:
current_format_data <- current_format_data %>% filter(!is.na(current_format_data$User.Type))

I can now set the datatypes for the features related to time to simplify downstream analysis.

In [46]:
current_format_data <- current_format_data %>% rename(Trip.Duration = Trip..Duration) %>% mutate(across(Trip.Duration, as.numeric))

In [47]:
current_format_data <- current_format_data %>% mutate(across(c(Start.Time, End.Time), ~mdy_hm(.x, tz = "America/Toronto")))

In [14]:
head(current_format_data)

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<chr>,<chr>,<dttm>,<chr>,<chr>,<dttm>,<chr>,<chr>,<chr>
4581278,1547,7021,2019-01-01 00:08:00,Bay St / Albert St,7233,2019-01-01 00:33:00,King / Cowan Ave - SMART,1296,Annual Member
4581279,1112,7160,2019-01-01 00:10:00,King St W / Tecumseth St,7051,2019-01-01 00:29:00,Wellesley St E / Yonge St (Green P),2947,Annual Member
4581280,589,7055,2019-01-01 00:15:00,Jarvis St / Carlton St,7013,2019-01-01 00:25:00,Scott St / The Esplanade,2293,Annual Member
4581281,259,7012,2019-01-01 00:16:00,Elizabeth St / Edward St (Bus Terminal),7235,2019-01-01 00:20:00,Bay St / College St (West Side) - SMART,283,Annual Member
4581282,281,7041,2019-01-01 00:19:00,Edward St / Yonge St,7257,2019-01-01 00:24:00,Dundas St W / St. Patrick St,1799,Annual Member
4581283,624,7041,2019-01-01 00:26:00,Edward St / Yonge St,7031,2019-01-01 00:36:00,Jarvis St / Isabella St,661,Annual Member


## Import and Cleaning 2: 2018 Data 

In [48]:
csvs_2018 <- list.files(path = "./bikeshare2018", pattern = "*.csv", full.names = TRUE)

In [49]:
data_format_2018 <- csvs_2018 %>% map(~read_csv(.x, name_repair = "universal", na = c("NA", "NULL", ""))) %>% reduce(rbind)
head(data_format_2018)

Rows: 178559 Columns: 9
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): trip_start_time, from_station_name, trip_stop_time, to_station_name...
dbl (4): trip_id, trip_duration_seconds, from_station_id, to_station_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 558370 Columns: 9
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): trip_start_time, from_station_name, trip_stop_time, to_station_name...
dbl (4): trip_id, trip_duration_seconds, from_station_id, to_station_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 822536 Columns: 9
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: 

trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
2383648,393,7018,1/1/2018 0:47,Bremner Blvd / Rees St,1/1/2018 0:54,7176,Bathurst St / Fort York Blvd,Annual Member
2383649,625,7184,1/1/2018 0:52,Ossington Ave / College St,1/1/2018 1:03,7191,Central Tech (Harbord St),Annual Member
2383650,233,7235,1/1/2018 0:55,Bay St / College St (West Side) - SMART,1/1/2018 0:59,7021,Bay St / Albert St,Annual Member
2383651,1138,7202,1/1/2018 0:57,Queen St W / York St (City Hall),1/1/2018 1:16,7020,Phoebe St / Spadina Ave,Annual Member
2383652,703,7004,1/1/2018 1:00,University Ave / Elm St,1/1/2018 1:12,7060,Princess St / Adelaide St E,Annual Member
2383653,1026,7078,1/1/2018 1:07,College St / Major St,1/1/2018 1:24,7130,Pears Av / Avenue Rd,Annual Member


From a quick inspection, the biggest difference from the current format is the lack of Bike ID information, which makes it impossible to produce bike-level insights before 2019. Otherwise features just need to be renamed and reordered and datatypes homogenized.  

In [50]:
new <- colnames(current_format_data)[-9]
old <- colnames(data_format_2018)
old_reindex <- c(1,2,3,4,5,7,6,8,9)
old <- old[order(old_reindex)]

In [51]:
data_2018_reformat <- data_format_2018 %>% rename_with(~new, all_of(old)) %>% 
    relocate(End.Station.Id, .before = End.Time)
head(data_2018_reformat)

Trip.Id,Trip.Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,User.Type
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
2383648,393,7018,1/1/2018 0:47,Bremner Blvd / Rees St,7176,1/1/2018 0:54,Bathurst St / Fort York Blvd,Annual Member
2383649,625,7184,1/1/2018 0:52,Ossington Ave / College St,7191,1/1/2018 1:03,Central Tech (Harbord St),Annual Member
2383650,233,7235,1/1/2018 0:55,Bay St / College St (West Side) - SMART,7021,1/1/2018 0:59,Bay St / Albert St,Annual Member
2383651,1138,7202,1/1/2018 0:57,Queen St W / York St (City Hall),7020,1/1/2018 1:16,Phoebe St / Spadina Ave,Annual Member
2383652,703,7004,1/1/2018 1:00,University Ave / Elm St,7060,1/1/2018 1:12,Princess St / Adelaide St E,Annual Member
2383653,1026,7078,1/1/2018 1:07,College St / Major St,7130,1/1/2018 1:24,Pears Av / Avenue Rd,Annual Member


After renaming, a check for missing data before correcting datatypes. No NAs, what a rare treat.

In [45]:
colSums(is.na(data_2018_reformat))

Trip.Id      Trip.Duration   Start.Station.Id         Start.Time 
                 0                  0                  0                  0 
Start.Station.Name     End.Station.Id           End.Time   End.Station.Name 
                 0                  0                  0                  0 
         User.Type 
                 0

Changing datatypes to match the current data format from 2019 onwards.

In [52]:
data_2018_reformat <- data_2018_reformat %>% mutate(across(c(Start.Station.Id, End.Station.Id), as.character))

In [53]:
data_2018_reformat <- data_2018_reformat %>% mutate(across(c(Start.Time, End.Time), ~mdy_hm(.x, tz = "America/Toronto")))

In [23]:
head(data_2018_reformat)

Trip.Id,Trip..Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,User.Type
<dbl>,<dbl>,<chr>,<dttm>,<chr>,<chr>,<dttm>,<chr>,<chr>
2383648,393,7018,2018-01-01 00:47:00,Bremner Blvd / Rees St,7176,2018-01-01 00:54:00,Bathurst St / Fort York Blvd,Annual Member
2383649,625,7184,2018-01-01 00:52:00,Ossington Ave / College St,7191,2018-01-01 01:03:00,Central Tech (Harbord St),Annual Member
2383650,233,7235,2018-01-01 00:55:00,Bay St / College St (West Side) - SMART,7021,2018-01-01 00:59:00,Bay St / Albert St,Annual Member
2383651,1138,7202,2018-01-01 00:57:00,Queen St W / York St (City Hall),7020,2018-01-01 01:16:00,Phoebe St / Spadina Ave,Annual Member
2383652,703,7004,2018-01-01 01:00:00,University Ave / Elm St,7060,2018-01-01 01:12:00,Princess St / Adelaide St E,Annual Member
2383653,1026,7078,2018-01-01 01:07:00,College St / Major St,7130,2018-01-01 01:24:00,Pears Av / Avenue Rd,Annual Member


For analysis across multiple years, I add a blank Bike ID feature to allow a proper union of tables

In [54]:
data_2018_compatible <- data_2018_reformat %>% add_column(Bike.Id = NA, .before = "User.Type")

## Import and Cleaning 3: 2017 Data

In [55]:
csvs_2017 <- list.files(path = "./2017 Data", pattern = "*.csv", full.names = TRUE)

In [56]:
data_format_2017 <- csvs_2017 %>% map(~read_csv(.x, name_repair = "universal", na = c("NA", "NULL", ""))) %>% reduce(rbind)

Rows: 132123 Columns: 9
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): trip_start_time, trip_stop_time, from_station_name, to_station_name...
dbl (4): trip_id, trip_duration_seconds, from_station_id, to_station_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 333353 Columns: 9
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): trip_start_time, trip_stop_time, from_station_name, to_station_name...
dbl (4): trip_id, trip_duration_seconds, from_station_id, to_station_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 663488 Columns: 7
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: 

ERROR: Error in rbind(deparse.level, ...): numbers of columns of arguments do not match


The big difference between 2017 and subsequent years is the quarterly CSVs do not all have the same format, so rbind throws an error. Loading them seperately reveals that Q3 and Q4 are missing the Station ID values to go with Station Name for both ends of a ride.

In [57]:
data_format_2017 <- csvs_2017 %>% map(~read_csv(.x, name_repair = "universal", na = c("NA", "NULL", "")))

Rows: 132123 Columns: 9
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): trip_start_time, trip_stop_time, from_station_name, to_station_name...
dbl (4): trip_id, trip_duration_seconds, from_station_id, to_station_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 333353 Columns: 9
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): trip_start_time, trip_stop_time, from_station_name, to_station_name...
dbl (4): trip_id, trip_duration_seconds, from_station_id, to_station_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 663488 Columns: 7
── Column specification ──────────────────────────────────────────────────────────────────
Delimiter: 

In [53]:
head(data_format_2017[[1]])

trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
712382,1/1/2017 0:00,1/1/2017 0:03,223,7051,Wellesley St E / Yonge St Green P,7089,Church St / Wood St,Member
712383,1/1/2017 0:00,1/1/2017 0:05,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member
712384,1/1/2017 0:05,1/1/2017 0:29,1394,7113,Parliament St / Aberdeen Ave,7199,College St W / Markham St,Member
712385,1/1/2017 0:07,1/1/2017 0:21,826,7077,College Park South,7010,King St W / Spadina Ave,Member
712386,1/1/2017 0:08,1/1/2017 0:12,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W,Member
712387,1/1/2017 0:12,1/1/2017 0:22,571,7021,Bay St / Albert St,7046,Niagara St / Richmond St W,Member


In [54]:
head(data_format_2017[[2]])

trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
868665,1/4/2017 0:01,1/4/2017 0:06,297,7073,Bremner Blvd / Spadina Ave,7211,Fort York Blvd / Garrison Rd,Member
868666,1/4/2017 0:07,1/4/2017 0:11,236,7038,Dundas St / Yonge St,7163,Yonge St / Wood St,Member
868667,1/4/2017 0:08,1/4/2017 0:14,380,7012,Elizabeth St / Edward St (Bus Terminal),7069,Queen St W / Spadina Ave,Member
868668,1/4/2017 0:12,1/4/2017 0:34,1299,7172,Strachan Ave / Princes' Blvd,7069,Queen St W / Spadina Ave,Member
868669,1/4/2017 0:13,1/4/2017 0:18,323,7039,Simcoe St / Dundas St W,7021,Bay St / Albert St,Member
868671,1/4/2017 0:18,1/4/2017 0:24,356,7155,Bathurst St / Lennox St,7199,College St W / Markham St,Member


In [55]:
head(data_format_2017[[3]])

trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1253914,7/1/2017 0:00,7/1/2017 0:15,910,Princess St / Adelaide St E,424 Wellington St W,Member
1253915,7/1/2017 0:01,7/1/2017 0:15,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
1253916,7/1/2017 0:01,7/1/2017 0:14,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
1253917,7/1/2017 0:01,7/1/2017 0:25,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
1253918,7/1/2017 0:01,7/1/2017 0:25,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
1253919,7/1/2017 0:03,7/1/2017 0:25,1332,Bay St / St. Joseph St,Niagara St / Richmond St W,Member


In [56]:
head(data_format_2017[[4]])

trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1971685,10/01/17 00:00:01,10/01/17 00:14:10,849,Queen St W / James St,Taddle Creek Park,Member
1971686,10/01/17 00:00:01,10/01/17 00:00:27,26,Stewart St / Bathurst St - SMART,Stewart St / Bathurst St - SMART,Casual
1971687,10/01/17 00:00:22,10/01/17 00:22:12,1310,Hayter St / Laplante Ave,Queen St W / Portland St,Member
1971688,10/01/17 00:00:38,10/01/17 00:11:09,631,Beverly St / College St,Beverly St / Dundas St W,Member
1971689,10/01/17 00:00:55,10/01/17 00:20:53,1198,Thompson St / Broadview Ave - SMART,Thompson St / Broadview Ave - SMART,Member
1971690,10/01/17 00:01:04,10/01/17 00:07:40,396,Dundonald St / Church St,Victoria St / Gould St (Ryerson University),Member


As an interim measure, adding blank columns to the Q3 and Q4 tables will facilitate a union of the full 2017 data. In the future, using a master table of Station Name/ID may permit identifying station ID for most/all stations.

In [58]:
data_format_2017[[3]] <- data_format_2017[[3]] %>% add_column(from_station_id = NA, .before = "from_station_name") %>%
add_column(to_station_id = NA, .before = "to_station_name")

In [59]:
data_format_2017[[4]] <- data_format_2017[[4]] %>% add_column(from_station_id = NA, .before = "from_station_name") %>%
add_column(to_station_id = NA, .before = "to_station_name")

In [60]:
data_format_2017 <- data_format_2017 %>% reduce(rbind)

The features are then renamed and reordered to match the current format from 2019 onwards.

In [61]:
new <- colnames(current_format_data)[-9]
old2 <- colnames(data_format_2017)
old2_reindex <- c(1,4,7,2,3,5,6,8,9)
old2 <- old2[order(old2_reindex)]

In [62]:
data_2017_reformat <- data_format_2017 %>% rename_with(~new, all_of(old2)) %>% select(c(1,4,5,2,6,7,3,8,9))

In [62]:
head(data_2017_reformat)

Trip.Id,Trip.Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,User.Type
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
712382,223,7051,1/1/2017 0:00,Wellesley St E / Yonge St Green P,7089,1/1/2017 0:03,Church St / Wood St,Member
712383,279,7143,1/1/2017 0:00,Kendal Ave / Bernard Ave,7154,1/1/2017 0:05,Bathurst Subway Station,Member
712384,1394,7113,1/1/2017 0:05,Parliament St / Aberdeen Ave,7199,1/1/2017 0:29,College St W / Markham St,Member
712385,826,7077,1/1/2017 0:07,College Park South,7010,1/1/2017 0:21,King St W / Spadina Ave,Member
712386,279,7079,1/1/2017 0:08,McGill St / Church St,7047,1/1/2017 0:12,University Ave / Gerrard St W,Member
712387,571,7021,1/1/2017 0:12,Bay St / Albert St,7046,1/1/2017 0:22,Niagara St / Richmond St W,Member


Checking for missing data shows that other than the missing station IDs from Q3/Q4, I find the expected missing data I added for Station IDs and 1 missing station name.

In [63]:
colSums(is.na(data_2017_reformat))

Trip.Id      Trip.Duration   Start.Station.Id         Start.Time 
                 0                  0            1026893                  0 
Start.Station.Name     End.Station.Id           End.Time   End.Station.Name 
                 0            1026893                  0                  1 
         User.Type 
                 0

The row with a missing station name appears to be some sort of data collection failure on a ride, as the end station information is not logged and the duration is 0 minutes. There is also an unusual encoding for End Time (NULLNULL). This encoding of NA does not appear anywhere else in the dataset so I replace this sole instance with NA.

In [64]:
data_2017_reformat %>% filter(is.na(End.Station.Name))

Trip.Id,Trip.Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,User.Type
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
2302635,0,NA,11/29/17 05:53:54,Seaton St / Dundas St E,NA,NULLNULL,NA,Casual


In [65]:
data_2017_reformat %>% filter(if_any(.fns = ~.x == "NULLNULL"))

Warning message:
“Using `if_any()` without supplying `.cols` was deprecated in dplyr 1.1.0.
ℹ Please supply `.cols` instead.”


Trip.Id,Trip.Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,User.Type
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
2302635,0,NA,11/29/17 05:53:54,Seaton St / Dundas St E,NA,NULLNULL,NA,Casual


In [63]:
data_2017_reformat <- data_2017_reformat %>% mutate(End.Time = replace(End.Time, End.Time == "NULLNULL", NA))

I then convert the station IDs to character datatype and time features to a datetime datatype. This is somewhat complicated by the fact that across the 4 quarters of 2017 there are 3 different date formats used, but lubridate is able to handle this by providing a range of formats for the parser to consider.

In [64]:
data_2017_reformat <- data_2017_reformat %>% mutate(across(c(Start.Station.Id, End.Station.Id), as.character)) %>%
    mutate(across(c(Start.Time, End.Time), ~parse_date_time(.x, c("dmy_HM", "mdy_HM", "mdy_HMS"), tz = "America/Toronto")))

Finally I recode the User Type field to match the coding used from 2018 onwards.

In [65]:
data_2017_reformat <- data_2017_reformat %>% mutate(User.Type = recode(User.Type, 
                                                     "Member" = "Annual Member",
                                                     "Casual" = "Casual Member"))

For analysis across multiple years, I add a blank Bike ID column to allow a proper union of tables

In [66]:
data_2017_compatible <- data_2017_reformat %>% add_column(Bike.Id = NA, .before = "User.Type")

## Import and Cleaning 4: 2017-2022 Data

In [67]:
data_all_years <- bind_rows(current_format_data, data_2018_compatible, data_2017_compatible)

In [71]:
head(data_all_years)

Trip.Id,Trip.Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<dbl>,<chr>,<dttm>,<chr>,<chr>,<dttm>,<chr>,<chr>,<chr>
4581278,1547,7021,2019-01-01 00:08:00,Bay St / Albert St,7233,2019-01-01 00:33:00,King / Cowan Ave - SMART,1296,Annual Member
4581279,1112,7160,2019-01-01 00:10:00,King St W / Tecumseth St,7051,2019-01-01 00:29:00,Wellesley St E / Yonge St (Green P),2947,Annual Member
4581280,589,7055,2019-01-01 00:15:00,Jarvis St / Carlton St,7013,2019-01-01 00:25:00,Scott St / The Esplanade,2293,Annual Member
4581281,259,7012,2019-01-01 00:16:00,Elizabeth St / Edward St (Bus Terminal),7235,2019-01-01 00:20:00,Bay St / College St (West Side) - SMART,283,Annual Member
4581282,281,7041,2019-01-01 00:19:00,Edward St / Yonge St,7257,2019-01-01 00:24:00,Dundas St W / St. Patrick St,1799,Annual Member
4581283,624,7041,2019-01-01 00:26:00,Edward St / Yonge St,7031,2019-01-01 00:36:00,Jarvis St / Isabella St,661,Annual Member


There are a series of outstanding missing data problems still to fix, but at this juncture they do not inhibit use of the dataset for most analysis with sensible provisions like removal of relevant NAs in code for plotting.

In [72]:
colSums(is.na(data_all_years))

Trip.Id      Trip.Duration   Start.Station.Id         Start.Time 
                 0                 16            1026893                  0 
Start.Station.Name     End.Station.Id           End.Time   End.Station.Name 
            187939            1031896                  1             193364 
           Bike.Id          User.Type 
           3415600                  0

In all rides there is information on either the ID or name of the start station, so hopefully the master list of stations can be used to fill in both features eventually. For now, the majority of analyses using the starting station should still be possible.

In [73]:
data_all_years %>% filter(is.na(Start.Station.Id) & is.na(Start.Station.Name))

Trip.Id,Trip.Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<dbl>,<chr>,<dttm>,<chr>,<chr>,<dttm>,<chr>,<chr>,<chr>


There are a non-trivial number of rides where there is no information at all about the end station but these can still be used for analyses focused on start station and duration. For the vast majority of rides, however, the missing information should be possible to reconstruct using a master list of stations

In [74]:
data_all_years %>% filter(is.na(End.Station.Id) & is.na(End.Station.Name))

Trip.Id,Trip.Duration,Start.Station.Id,Start.Time,Start.Station.Name,End.Station.Id,End.Time,End.Station.Name,Bike.Id,User.Type
<dbl>,<dbl>,<chr>,<dttm>,<chr>,<chr>,<dttm>,<chr>,<chr>,<chr>
5370500,696,7228,2019-06-17 13:21:00,Queen St W / Roncesvalles Ave,NA,2019-06-17 13:32:00,NA,2345,Annual Member
5679465,0,7077,2019-07-11 16:45:00,College Park South,NA,2019-07-11 16:45:00,NA,1232,Casual Member
6033723,327,7444,2019-08-06 01:41:00,Clendenan Ave / Rowland St - SMART,NA,2019-08-06 01:46:00,NA,1890,Casual Member
6735107,979,7432,2019-09-25 14:48:00,Frederick St / King St E,NA,2019-09-25 15:04:00,NA,2113,Annual Member
6746731,60,7077,2019-09-26 13:31:00,College Park South,NA,2019-09-26 13:32:00,NA,1430,Annual Member
6750357,300,7104,2019-09-26 17:54:00,King St E / River St,NA,2019-09-26 17:59:00,NA,3478,Annual Member
6752509,1112,7382,2019-09-26 20:13:00,Simcoe St / Adelaide St W,NA,2019-09-26 20:32:00,NA,530,Annual Member
6752610,1620,7076,2019-09-26 20:24:00,York St / Queens Quay W,NA,2019-09-26 20:51:00,NA,1557,Annual Member
6754350,1822,7471,2019-09-27 06:32:00,Lake Shore Blvd W / Louisa St,NA,2019-09-27 07:02:00,NA,3176,Annual Member


Finally, I write out a CSV to use as input for downstream analysis.

In [69]:
write_csv(data_all_years, "../data_all_years.csv")